## DP05. 按小时（集合）批量统计词频并保存对应变量
[Back to HOME](https://visforever.github.io/)

### 批量统计词频（多线程 vs 常规）
* 词频统计(按小时，i.e., 按集合统计)
* 处理24小时的词频，提取每小时top200的高频词，并保存在文件中
* `存疑`：感觉多线程也没有很快，有待分析...
* `疑惑已解决`：见[DP07. 多进程处理程序（替换多线程 - DP04）](https://visforever.github.io/blog/blogNotes/dp07_multiProcess.html)

In [1]:
import pickle
# Input: 变量名，保存路径
# Output: 保存路径
def save_variable1(v, filename):
    f = open(filename, 'wb')
    pickle.dump(v, f, 0)  # 在dump()里加上第三个参数，设为0(ASCII协议).
    f.close()
    return filename

# Input: 保存的变量路径
# Output: 变量
def load_variable1(filename):
    f = open(filename, 'rb')
    r = pickle.load(f)
    f.close()
    return r

def save_variable2(v, filename):
    try: 
        # f = open('test001.txt','w',encoding='utf-8') #open 打开要写入的文件，'w'就是写入的意思，如果有中文encoding进行编码
        with open(filename,'w',encoding='utf-8') as f:
            f.write(v)  # write是写入的意思，将（）中的内容写入到f文件里
            
    # 无论try中的程序是否存在报错，则都执行下面的关闭
    finally: 
        if f:  # 判断下f是否打开了，如果没打开则不需要关闭，打开了则关闭
            f.close()
    return filename

def load_variable2(filename):
    # 首先try一下，如果程序打开以及在打开后读取一系列操作后有报错，则不中断程序
    try:  
        # f = open(filename, 'r', encoding='utf-8')
        with open(filename, 'r', encoding='utf-8') as f:
            strContent = f.read()
    # 无论try中的程序是否存在报错，则都执行下面的关闭
    finally:  
        if f:  # 判断下f是否打开了，如果没打开则不需要关闭，打开了则关闭
            f.close()
    return strContent

In [2]:
import pymongo
con_str = "localhost:27017"
db = pymongo.MongoClient(con_str, serverSelectionTimeoutMS = 10000)
try:
    # print(db.server_info())
    
    # 查询数据库中所有数据库的名称
    db_names = db.list_database_names()
    print(db_names)
except Exception:
    print("connect failed!")
    
targetDB = 'db_Weibo_SH0331'

if targetDB in db_names:
    db_Weibo_SH0331 = db['db_Weibo_SH0331']
    print(targetDB + ": 已连接！")

['admin', 'config', 'db_Weibo_SH0331', 'db_Weibo_SH_0331_All', 'db_tvcg3years', 'db_tvcg4years', 'db_tvcgR3', 'db_tvcg_28years', 'db_weiboDB', 'local', 'test', 'weiboInfoAll']
db_Weibo_SH0331: 已连接！


In [3]:
# key：集合的名称
# value: 集合中所有词的数量，不一样的词的数量，词频字典

# wordFreqDict = {
#     'col_weibo_timePeriod0' : [17104459, 63090, {'抗疫': 77797, '11': 75271, '12345': 74553, ...}]
#     'col_weibo_timePeriod1' : [14104454, 6308, {'疫情': 77797, '11': 75271, '12345': 74553, ...}]
# }

def computeWordFreq_v1(targetCol, wordFreqDict, per = 1.0):
    
    wordFreqDict[targetCol] = list()
    
    exec(targetCol + " = db_Weibo_SH0331['" + targetCol+ "']")
    exec("recs = " + targetCol + ".find()")
    loc = locals()
    
    tempCnt_Words = 0  # ●1 存放当前集合中的词的总数

    freqWords = {}  # key(词汇)，value(词频数)

    for cW in loc['recs']:
        # 手动写词频统计函数，基于字典实现
        currentContent = cW.get('kWords')
        tempCnt_Words = tempCnt_Words + len(currentContent)  # 存放当前集合中关键词的总数量

        for eachWord in currentContent:
            freqWords[eachWord] = freqWords.get(eachWord, 0) + 1  # 统计每个关键词的词频

    items = list(freqWords.items())  # 转换为list
    total_num = len(items)  # ●2 不一样的词的个数
    # print("items.len = " + str(total_num)）  # ●2 不一样的词的个数
    items.sort(key=lambda x: x[1], reverse=True)  # 按照词频排序
    
    topK = int(total_num * per)

    wordcloudDic = dict()  # ●3 存放前k个词 & 词频
    for i in range(topK):
        word, count = items[i]
        wordcloudDic[word] = count
        # print("{0:<20}{1:>10}".format(word, count))
        
    wordFreqDict[targetCol].append(tempCnt_Words)
    wordFreqDict[targetCol].append(len(items))
    wordFreqDict[targetCol].append(wordcloudDic)



In [4]:
# 没用多线程，没用多进程
import time

if __name__ == '__main__':
    startTime = time.time() 
    
    # 循环操作24个集合：新建指定字段
    subCol24 = list()
    subCol24.append(["col_weibo_timePeriod_" + str(period) for period in range(0, 24)])
    print(subCol24[0])
    colPackages = subCol24[0]

    wordFreqDict_Direct = dict()
    per = 0.2  # 返回的是前200个高频词
    
    for i in range(0, len(colPackages)):
        print(colPackages[i])
        computeWordFreq_v1(colPackages[i], wordFreqDict_Direct, per)
        
    endTime = time.time()    
    print("---------------> Time consuming: %.2f" % (endTime - startTime))
    save_variable1(wordFreqDict_Direct, 'wordFreqDict_Direct_WithType.txt')
    save_variable2(str(wordFreqDict_Direct), 'wordFreqDict_Direct_WithoutType.txt')  # 只能查看字符串

['col_weibo_timePeriod_0', 'col_weibo_timePeriod_1', 'col_weibo_timePeriod_2', 'col_weibo_timePeriod_3', 'col_weibo_timePeriod_4', 'col_weibo_timePeriod_5', 'col_weibo_timePeriod_6', 'col_weibo_timePeriod_7', 'col_weibo_timePeriod_8', 'col_weibo_timePeriod_9', 'col_weibo_timePeriod_10', 'col_weibo_timePeriod_11', 'col_weibo_timePeriod_12', 'col_weibo_timePeriod_13', 'col_weibo_timePeriod_14', 'col_weibo_timePeriod_15', 'col_weibo_timePeriod_16', 'col_weibo_timePeriod_17', 'col_weibo_timePeriod_18', 'col_weibo_timePeriod_19', 'col_weibo_timePeriod_20', 'col_weibo_timePeriod_21', 'col_weibo_timePeriod_22', 'col_weibo_timePeriod_23']
col_weibo_timePeriod_0
col_weibo_timePeriod_1
col_weibo_timePeriod_2
col_weibo_timePeriod_3
col_weibo_timePeriod_4
col_weibo_timePeriod_5
col_weibo_timePeriod_6
col_weibo_timePeriod_7
col_weibo_timePeriod_8
col_weibo_timePeriod_9
col_weibo_timePeriod_10
col_weibo_timePeriod_11
col_weibo_timePeriod_12
col_weibo_timePeriod_13
col_weibo_timePeriod_14
col_weibo_t